In [ ]:
# Reading the dataset
# Get the current working directory (cwd)
cwd = os.getcwd()
# Get all the files in that directory
files = os.listdir(cwd)
print("Files in %r: %s" % (cwd, files))

#Reading the GLOBAL TEMPERATURE BY CITY CSV
data = pd.read_csv("/content/GlobalLandTemperaturesByCity.csv.zip")
data

NameError: name 'os' is not defined

In [ ]:
# Reading the dataset
# Import the os module
import os
# Get the current working directory (cwd)
cwd = os.getcwd()
# Get all the files in that directory
files = os.listdir(cwd)
print("Files in %r: %s" % (cwd, files))

# Import the pandas module
import pandas as pd
#Reading the GLOBAL TEMPERATURE BY CITY CSV
data = pd.read_csv("/content/GlobalLandTemperaturesByCity.csv.zip")
data

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [ ]:
# Gives the Basic Structure of the Data Frame such as Data type and Col names.
data.info()

# Show the basic Statistical Values of the data such as Mean, Count, StD, Min and Quartiles
data.describe()

# Shape of the dataset
data.shape

# Names of the Column
data.columns

# First 5 Records in the dataset
data.head()

# Last 5 Records in the dataset
data.tail()

# Here we can see the unique values exist in each column
data.nunique()

# Shows the number of Null values in each column
data.isnull().sum()

In [ ]:
# Dropping all null values
data = data.dropna(how='any' ,axis=0)
data.shape

In [ ]:
# Converting Dt into Date
data['Date'] = pd.to_datetime(data['Date'])
# Setting the Index
data.set_index('Date',inplace = True)
data.index

In [ ]:
# Now we use year as index
data['Year']= data.index.year
data.head()

In [ ]:
delhi = data.loc[data['City'] == ('New Delhi' or "Delhi"), ['Date','AverageTemperature']]
delhi.columns = ['Date','Temp']

delhi['Date'] = pd.to_datetime(delhi['Date'])
delhi.reset_index(drop=True, inplace=True)
delhi.set_index('Date', inplace=True)
delhi

NameError: name 'data' is not defined

In [ ]:
plt.figure(figsize=(22,6))
sns.lineplot(x=delhi.index, y=delhi['Temp'])
plt.title('Temperature Variation in Delhi from 1980 until 2013')
plt.xlabel("Date",fontsize=14)
plt.ylabel("Temperature in C'",fontsize=14)
plt.legend()
plt.savefig(f"./DataViz/Delhi/1980-2013Temp.jpeg") # To Save the Plot as an JPEG Image
plt.show()

In [ ]:
pivot = pd.pivot_table(delhi, values='Temp', index='Month', columns='Year', aggfunc='mean')
pivot.plot(figsize=(20,6),colormap="coolwarm")
plt.title('Yearly Delhi temperatures')
plt.xlabel('Months')
plt.ylabel('Temperatures')

plt.xticks([x for x in range(1,13)])
plt.legend().remove()
plt.savefig(f"./DataViz/Delhi/TemperatureChangeDelhi.png")
plt.show()

In [ ]:

monthly_seasonality = pivot.mean(axis=1)
monthly_seasonality.plot(figsize=(20,6))
plt.title('Monthly Temperatures in Delhi')
plt.xlabel('Months',fontsize=14)
plt.ylabel('Temperature',fontsize=14)
plt.xticks([x for x in range(1,13)])
plt.savefig(f"./DataViz/Delhi/AvgTempChangeMonth.png")
plt.show()

In [ ]:

year_avg = pd.pivot_table(delhi, values='Temp', index='Year', aggfunc='mean')
# The rolling mean of Temp in last 10 years
# Rolling mean window is 3
year_avg['10 Years MA'] = year_avg['Temp'].rolling(3).mean()
year_avg[['Temp','10 Years MA']].plot(kind="line",
                                      figsize=(20,6),
                                      colormap="cool",
                                      marker='o',)

plt.title('Yearly AVG Temperatures in Delhi')
plt.xlabel('Years')
plt.ylabel('Temperature')
plt.xticks([x for x in range(1980,2012,2)])

plt.savefig(f"./DataViz/Delhi/TempChangeVsRollingMean.png")
plt.show()

In [ ]:
# Train Size is 200
df_train = delhi[-300:]
# Val size is 100
df_val = delhi[200:300]
# Test Size is 100
df_test = delhi[:-200]

In [ ]:
from statsmodels.tsa.stattools import adfuller

print('Augmented Ducky Fuller Test Results')

test_data = adfuller(df_train.iloc[:,0].values,autolag='AIC')

data_output = pd.Series(test_data[0:4],index=['Test Statistic','p-value','Lags Used','Number of Observation Used'])

for key,value in test_data[4].items():

    data_output['Critical value (%s)'%key] = value

print(data_output)

In [ ]:
import math
def check_stationarity(y, lags_plots=48, figsize=(22,8)):

    # Creating plots of the DF
    y = pd.Series(y)
    fig = plt.figure()

    ax1 = plt.subplot2grid((3, 3), (0, 0), colspan=2)
    ax2 = plt.subplot2grid((3, 3), (1, 0))
    ax3 = plt.subplot2grid((3, 3), (1, 1))
    ax4 = plt.subplot2grid((3, 3), (2, 0), colspan=2)

    y.plot(ax=ax1, figsize=figsize)

    ax1.set_title('Delhi Temperature Variation')
    plot_acf(y, lags=lags_plots, zero=False, ax=ax2);
    plot_pacf(y, lags=lags_plots, zero=False, ax=ax3);
    sns.distplot(y, bins=int(sqrt(len(y))), ax=ax4)
    ax4.set_title('Distribution Chart')

    plt.tight_layout()

    plt.savefig(f"./DataViz/Delhi/AdfullerTest.png")

    print('Results of Dickey-Fuller Test:')
    adfinput = adfuller(y)
    adftest = pd.Series(adfinput[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    adftest = round(adftest,4)

    for key, value in adfinput[4].items():
        adftest["Critical Value (%s)"%key] = value.round(4)

    print(adftest)

    if adftest[0].round(2) < adftest[5].round(2):
        print('\nThe Test Statistics is lower than the Critical Value of 5%.\nThe serie seems to be stationary')
    else:
        print("\nThe Test Statistics is higher than the Critical Value of 5%.\nThe serie isn't stationary")

# The first approach is to check the series without any transformation
check_stationarity(df_train["Temp"])


NameError: name 'df_train' is not defined

In [ ]:
#Now we break the data into sub section
from statsmodels.tsa.seasonal import seasonal_decompose

decomp= seasonal_decompose(df_train["Temp"],period=3)
trend = decomp.trend
seasonal = decomp.seasonal
residual = decomp.resid

plt.subplot(411)
plt.figure(figsize=(20,8))
plt.plot(df_train["Temp"],'r-')
plt.xlabel('Original')
plt.figure(figsize=(20,8))

plt.subplot(412)
plt.plot(trend,'k-')
plt.xlabel('Trend')
plt.figure(figsize=(20,8))

plt.subplot(413)
plt.plot(seasonal,'g-')
plt.xlabel('Seasonal')
plt.figure(figsize=(20,8))

plt.subplot(414)
plt.plot(residual,'y-')
plt.xlabel('Residual')
plt.figure(figsize=(20,8))

plt.tight_layout()

In [ ]:
check_stationarity(df_train['Temp'].diff(12).dropna())

In [ ]:
Interactive Tableau Dashboard
You can access the Tableau Dashboard to learn more about the pattern and trends in the Dataset in more interactive and fun manner.

https://public.tableau.com/views/ClimateChangeAnalysisDashboardDA2/Dashboard1?:language=en-US&:display_count=n&:origin=viz_share_link